In [40]:
from pulp import *
import pandas as pd

# 加载数据
file_path = r'C:\Users\zhangyutang\Desktop\附件3_处理_final_modified.xlsx'
try:
    df = pd.read_excel(file_path)
    print("数据成功加载！")
    print(df.head())
except FileNotFoundError:
    print(f"错误：文件未找到，请检查路径：{file_path}")
except Exception as e:
    print(f"加载数据时发生错误：{e}")

# 提取餐次信息，方便后续按餐次划分食物
meals = df['餐次'].unique()
food_items = df.index.tolist() # 使用索引作为食物的唯一标识

# 必需氨基酸列表 (请务必核对你的数据列名)
essential_amino_acids = [
    "异亮氨酸 (mg/100g)",
    "亮氨酸 (mg/100g)",
    "赖氨酸 (mg/100g)",
    "含硫氨基酸(SAA)_蛋氨酸 (mg/100g)", # 注意：这里只使用了蛋氨酸，如果你的模型需要考虑总含硫氨基酸，请调整
    "芳香族氨基酸(AAA)_苯丙氨酸 (mg/100g)", # 注意：这里只使用了苯丙氨酸，如果你的模型需要考虑总芳香族氨基酸，请调整
    "苏氨酸 (mg/100g)",
    "色氨酸 (mg/100g)",
    "缬氨酸 (mg/100g)"
]

# 每日膳食蛋白质参考摄入量 (Pref) - 单位为克
protein_reference = 30

# 能量上下限 (±10% 容差)
energy_lower = 2160 * 0.9
energy_upper = 2640 * 1.1

# 宏量营养素供能占比
protein_energy_ratio = [0.10, 0.15]
fat_energy_ratio = [0.20, 0.30]
carb_energy_ratio = [0.50, 0.65]

# 宏量营养素摄入总量 (±10% 容差)
protein_lower = 57 * 0.9
protein_upper = 94.5 * 1.1
fat_lower = 50.6 * 0.9
fat_upper = 84 * 1.1
carb_lower = 285 * 0.9
carb_upper = 409.5 * 1.1

# 微量营养素摄入上下限 (±10% 容差)
micronutrient_bounds = {
    "钙 (mg/100g)": [760 * 0.9, 840 * 1.1],
    "铁 (mg/100g)": [11.4 * 0.9, 12.6 * 1.1],
    "锌 (mg/100g)": [11.875 * 0.9, 13.125 * 1.1],
    "维生素C (mg/100g)": [95 * 0.9, 105 * 1.1],
    "硫胺素 (μg/100g)": [1.33 * 1000 * 0.9, 1.47 * 1000 * 1.1], # 换算为 mg
    "核黄素 (mg/100g)": [1.33 * 0.9, 1.47 * 1.1],
    "总维生素A (μg/100g)": [760 * 0.9, 840 * 1.1], # 使用总维生素A作为代表
}

# 最大份数限制 (假设一个合理的上限)
max_servings = 5

# 创建优化模型
model = LpProblem("Diet Optimization with AAS", LpMaximize)

# 定义决策变量
x = LpVariable.dicts("servings", food_items, 0, max_servings, LpInteger)
y = LpVariable.dicts("chosen", food_items, 0, 1, LpBinary)
zb = LpVariable("AAS_breakfast", 0)
zl = LpVariable("AAS_lunch", 0)
zd = LpVariable("AAS_dinner", 0)

# 目标函数
model += zb + zl + zd, "Total_AAS_Score"

# 链接约束: xi > 0 => yi = 1
for i in food_items:
    model += x[i] <= max_servings * y[i]

# 食物结构多样性约束
model += lpSum(y[i] for i in food_items) >= 12, "Min_12_food_types_per_day"
for meal in meals:
    meal_items = df[df['餐次'] == meal].index.tolist()
    model += lpSum(y[i] for i in meal_items) <= 7, f"Max_7_food_types_per_{meal}"

# 总能量摄入约束
model += lpSum(x[i] * df.loc[i, '可食部\n（克/份）'] * df.loc[i, '能量 (kcal/100g)'] / 100 for i in food_items) >= energy_lower, "Lower_bound_on_total_energy"
model += lpSum(x[i] * df.loc[i, '可食部\n（克/份）'] * df.loc[i, '能量 (kcal/100g)'] / 100 for i in food_items) <= energy_upper, "Upper_bound_on_total_energy"

# 宏量营养素供能占比约束 (需要计算每种宏量营养素的总能量)
total_protein_energy = lpSum(x[i] * df.loc[i, '可食部\n（克/份）'] * df.loc[i, '蛋白质 (g/100g)'] * 4 / 100 for i in food_items)
total_fat_energy = lpSum(x[i] * df.loc[i, '可食部\n（克/份）'] * df.loc[i, '脂肪 (g/100g)'] * 9 / 100 for i in food_items)
total_carb_energy = lpSum(x[i] * df.loc[i, '可食部\n（克/份）'] * df.loc[i, '碳水化合物 (g/100g)'] * 4 / 100 for i in food_items)
total_energy = lpSum(x[i] * df.loc[i, '可食部\n（克/份）'] * df.loc[i, '能量 (kcal/100g)'] / 100 for i in food_items)

model += total_protein_energy >= protein_energy_ratio[0] * total_energy, "Lower_bound_on_protein_energy_ratio"
model += total_protein_energy <= protein_energy_ratio[1] * total_energy, "Upper_bound_on_protein_energy_ratio"
model += total_fat_energy >= fat_energy_ratio[0] * total_energy, "Lower_bound_on_fat_energy_ratio"
model += total_fat_energy <= fat_energy_ratio[1] * total_energy, "Upper_bound_on_fat_energy_ratio"
model += total_carb_energy >= carb_energy_ratio[0] * total_energy, "Lower_bound_on_carb_energy_ratio"
model += total_carb_energy <= carb_energy_ratio[1] * total_energy, "Upper_bound_on_carb_energy_ratio"

# 宏量营养素摄入总量约束
total_protein = lpSum(x[i] * df.loc[i, '可食部\n（克/份）'] * df.loc[i, '蛋白质 (g/100g)'] / 100 for i in food_items)
total_fat = lpSum(x[i] * df.loc[i, '可食部\n（克/份）'] * df.loc[i, '脂肪 (g/100g)'] / 100 for i in food_items)
total_carb = lpSum(x[i] * df.loc[i, '可食部\n（克/份）'] * df.loc[i, '碳水化合物 (g/100g)'] / 100 for i in food_items)

model += total_protein >= protein_lower, "Lower_bound_on_total_protein"
model += total_protein <= protein_upper, "Upper_bound_on_total_protein"
model += total_fat >= fat_lower, "Lower_bound_on_total_fat"
model += total_fat <= fat_upper, "Upper_bound_on_total_fat"
model += total_carb >= carb_lower, "Lower_bound_on_total_carbohydrates"
model += total_carb <= carb_upper, "Upper_bound_on_total_carbohydrates"

# 微量营养素摄入上下限约束
for nutrient, bounds in micronutrient_bounds.items():
    if nutrient in df.columns:
        lower_bound = bounds[0] * 0.9 if bounds[0] != 0 else 0
        upper_bound = bounds[1] * 1.1
        model += lpSum(x[i] * df.loc[i, '可食部\n（克/份）'] * df.loc[i, nutrient] / 100 for i in food_items) >= lower_bound, f"Lower_bound_on_total_{nutrient}"
        model += lpSum(x[i] * df.loc[i, '可食部\n（克/份）'] * df.loc[i, nutrient] / 100 for i in food_items) <= upper_bound, f"Upper_bound_on_total_{nutrient}"

# 氨基酸评分约束 (线性近似)
for meal in meals:
    meal_df = df[df['餐次'] == meal]
    meal_items_indices = meal_df.index.tolist()
    total_protein_per_meal = lpSum(x[i] * df.loc[i, '可食部\n（克/份）'] * df.loc[i, '蛋白质 (g/100g)'] / 100 for i in meal_items_indices)

    if meal == '早餐':
        aas_var = zb
    elif meal == '午餐':
        aas_var = zl
    else:
        aas_var = zd

    for amino_acid_col in essential_amino_acids:
        if amino_acid_col in df.columns and reference_protein.get(f"{amino_acid_col.split('(')[0].strip()} (mg/g protein)") is not None:
            amino_acid_intake = lpSum(x[i] * df.loc[i, '可食部\n（克/份）'] * df.loc[i, amino_acid_col] / 100 for i in meal_items_indices)
            ref_value = reference_protein[f"{amino_acid_col.split('(')[0].strip()} (mg/g protein)"]
            # 将参考蛋白的单位从 mg/g 转换为 mg/30g (Pref)
            ref_value_for_30g = ref_value * protein_reference
            # 线性近似约束：AAS <= (该氨基酸摄入量) / (参考蛋白中该氨基酸含量)
            # 注意这里为了避免除以零，添加了一个很小的正数
            model += aas_var <= (amino_acid_intake / (ref_value_for_30g + 1e-6)), f"AAS_constraint_for_{amino_acid_col.split('(')[0].strip()}_in_{meal}"

# 求解模型
try:
    model.solve()
    print("模型求解状态:", LpStatus[model.status])

    if model.status == 'LpOptimal':
        print("\n优化结果:")
        total_aas = value(zb) + value(zl) + value(zd)
        print(f"总 AAS 评分: {total_aas:.2f}")

        selected_foods = {}
        for i in food_items:
            if value(x[i]) > 1e-6:
                meal = df.loc[i, '餐次']
                food_name = df.loc[i, '食物名称']
                servings = value(x[i])
                if meal not in selected_foods:
                    selected_foods[meal] = []
                selected_foods[meal].append(f"{food_name} ({servings:.1f} 份)")

        for meal, foods in selected_foods.items():
            print(f"\n{meal}:")
            for food in foods:
                print(f"- {food}")

    else:
        print("模型没有找到最优解。")

except Exception as e:
    print(f"求解模型时发生错误: {e}")

数据成功加载！
   餐次  序号 食物名称 主要成分     食物编码  可食部\n（克/份）  价格\n（元/份）  是否\n可半份  水分 (g/100g)  \
0  早餐   1   牛奶   牛奶  101101x         200        1.5        0         87.6   
1  早餐   2   酸奶   酸奶   103001         125        1.5        0         85.5   
2  早餐   3   豆浆   黄豆    31101          10        1.5        0         10.2   
3  早餐   4  大米粥   稻米  012001x          15        0.5        0         13.3   
4  早餐   5  小米粥   小米    15101          15        0.5        0         11.6   

   能量 (kcal/100g)  ...  色氨酸 (mg/100g)  缬氨酸 (mg/100g)  精氨酸 (mg/100g)  \
0              65  ...             54            178             94   
1              70  ...             48            155             95   
2             390  ...            455           1726           2840   
3             346  ...            124            426            570   
4             361  ...            178            483            315   

   组氨酸 (mg/100g)  丙氨酸 (mg/100g)  天冬氨酸 (mg/100g)  谷氨酸 (mg/100g)  甘氨酸 (mg/100g)  \
0             78     

E:\anaconda\envs\py310\lib\site-packages\pulp\pulp.py:1424: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


模型求解状态: Infeasible
模型没有找到最优解。
